<a href="https://colab.research.google.com/github/sohag221/CNN-model-trained-with-medicinal-leaf./blob/main/EfficientNetB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mdsohag2165_bd_medicinal_plant_path = kagglehub.dataset_download('mdsohag2165/bd-medicinal-plant')

print('Data source import complete.')


In [ ]:
import os, time, copy, shutil, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Define paths
SOURCE_DIR = "/kaggle/input/bd-medicinal-plant/Medicinal Plant BD"
DEST_DIR = "/kaggle/working/medicinal_split"
os.makedirs(DEST_DIR, exist_ok=True)

# Ratios
train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

# Create folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(DEST_DIR, split), exist_ok=True)

# Split images per class
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path): continue
    images = os.listdir(class_path)
    random.shuffle(images)
    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    train_imgs = images[:n_train]
    val_imgs = images[n_train:n_train+n_val]
    test_imgs = images[n_train+n_val:]

    for split, split_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
        split_dir = os.path.join(DEST_DIR, split, class_name)
        os.makedirs(split_dir, exist_ok=True)
        for img in tqdm(split_imgs, desc=f"{split} - {class_name}"):
            shutil.copy2(os.path.join(class_path, img), os.path.join(split_dir, img))


In [ ]:
IMG_SIZE = 300  # For EfficientNet-B3
BATCH_SIZE = 24
NUM_EPOCHS = 10
PATIENCE = 5
NUM_CLASSES = 10  # You have 10 folders/classes

DATA_DIR = DEST_DIR
PREFIX = "/kaggle/working/efficientnetb3"

# Transforms
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tfms)
test_ds  = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)

class_names = train_ds.classes
print("Classes:", class_names)


In [ ]:
model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)


In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25, patience=5):
    best_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    tr_loss, val_loss, tr_acc, val_acc = [], [], [], []
    patience_cnt = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start = time.time()

        # Training
        model.train()
        run_loss, run_correct = 0.0, 0
        for x, y in tqdm(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            run_loss += loss.item() * x.size(0)
            run_correct += (out.argmax(1) == y).sum().item()
        tr_loss.append(run_loss / len(train_loader.dataset))
        tr_acc.append(run_correct / len(train_loader.dataset))

        # Validation
        model.eval()
        run_loss, run_correct = 0.0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)
                run_loss += loss.item() * x.size(0)
                run_correct += (out.argmax(1) == y).sum().item()
        val_loss.append(run_loss / len(val_loader.dataset))
        val_acc.append(run_correct / len(val_loader.dataset))

        print(f"Train Loss {tr_loss[-1]:.4f} Acc {tr_acc[-1]:.4f}")
        print(f"Val   Loss {val_loss[-1]:.4f} Acc {val_acc[-1]:.4f}")
        print(f"Time Taken: {(time.time() - start):.1f}s")

        # Early stopping
        if val_acc[-1] > best_acc:
            best_acc = val_acc[-1]
            best_wts = copy.deepcopy(model.state_dict())
            patience_cnt = 0
        else:
            patience_cnt += 1
            if patience_cnt >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_wts)
    return model, tr_loss, val_loss, tr_acc, val_acc, best_acc


In [ ]:
model, tr_loss, val_loss, tr_acc, val_acc, best_val_acc = train_model(
    model, criterion, optimizer, train_loader, val_loader,
    NUM_EPOCHS, PATIENCE
)


In [ ]:
epochs = range(1, len(tr_loss)+1)
fig, ax = plt.subplots(1, 2, figsize=(12,5))
ax[0].plot(epochs, tr_loss, label="Train"); ax[0].plot(epochs, val_loss, label="Val")
ax[0].set_title("Loss"); ax[0].legend()
ax[1].plot(epochs, tr_acc, label="Train"); ax[1].plot(epochs, val_acc, label="Val")
ax[1].set_title("Accuracy"); ax[1].legend()
plt.tight_layout()
plt.savefig(f"{PREFIX}_curves.png", dpi=300)
plt.show()


In [ ]:
model.eval()
test_probs, test_labels = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        out = model(x)
        probs = torch.softmax(out, dim=1)
        test_probs.append(probs.cpu())
        test_labels.extend(y.cpu().numpy())

test_probs = torch.cat(test_probs).numpy()
y_true = np.array(test_labels)
y_pred = np.argmax(test_probs, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
plt.figure(figsize=(8,6))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", cbar_kws={'label': 'Image Count'})
plt.title("Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.savefig(f"{PREFIX}_cm_counts.png", dpi=300)
plt.show()


In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load model

model.eval()

# Class names
class_names = ['Bhibitaki', 'Candelabra plant', 'Chebulic Myrobalan', 'Gotu Kola',
               'Holy Basil', 'Indian Borage', 'Lemongrass', 'Longevity Spinach',
               'Madagascar Periwinkle', 'Neem Tree']

# Transform (same as training)
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Make sure you set the right device first
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)  # Move image to the same device
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return class_names[predicted.item()], image


# Folder with test images
folder_path = '/kaggle/input/bd-medicinal-plant/Medicinal Plant BD/Bhibitaki'  # change as needed

# Loop through all images in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Predict and display results
for i, image_file in enumerate(image_files[:5]):  # limit to first 5 images for display
    image_path = os.path.join(folder_path, image_file)
    prediction, image = predict_image(image_path)

    plt.figure(figsize=(4, 4))
    plt.imshow(image)
    plt.title(f"Predicted: {prediction}")
    plt.axis('off')
    plt.show()


In [ ]:
# Save model state dictionary
torch.save(model.state_dict(), 'efficientnet_b3_model.pth')
